<a href="https://colab.research.google.com/github/pacomm5/Plantillas/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clonamos el repositorio para obtener los dataSet

In [1]:
#!git clone https://github.com/joanby/machinelearning-az.git
! git clone https://github.com/pacomm5/machinelearning-az 

Cloning into 'machinelearning-az'...
remote: Enumerating objects: 10541, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 10541 (delta 25), reused 12 (delta 5), pack-reused 10505
Receiving objects: 100% (10541/10541), 311.58 MiB | 26.99 MiB/s, done.
Resolving deltas: 100% (245/245), done.
Checking out files: 100% (10250/10250), done.


# Damos acceso a nuestro Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [3]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependendias

In [4]:
!pip install sklearn

# Las instrucciones de instalación se pueden consultar en http://xgboost.readthedocs.io/en/latest/build.html


In [5]:
!pip install xgboost

# XGBoost

# Cómo importar las librerías


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [7]:
dataset = pd.read_csv('/content/machinelearning-az/datasets/Part 10 - Model Selection & Boosting/Section 49 - XGBoost/Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [8]:
dataset.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Codificar datos categóricos

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [2])],   
    remainder='passthrough'                        
)
X = np.array(ct.fit_transform(X), dtype=np.float)
X = X[:, 1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':


# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Ajustar el modelo XGBoost al Conjunto de Entrenamiento

In [11]:
from xgboost import XGBClassifier

In [12]:
classifier = XGBClassifier() # ponemos los valores que vienen por defecto
classifier.fit(X_train, y_train) # aqui lo ajustamos. 

XGBClassifier()

# Predicción de los resultados con el Conjunto de Testing

In [13]:
y_pred  = classifier.predict(X_test)

In [18]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

# Elaborar una matriz de confusión

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [16]:
cm

array([[1534,   61],
       [ 208,  197]])

In [17]:
(1534+197)/(1534+61+208+197) # esta es la precisión no hemos mejorado con el modelo anterior pero es mucho más rapido. 

0.8655

# Aplicar k-fold cross validation



In [15]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
accuracies.mean()
accuracies.std() # es inferior al 1% es decir bastante interesante. 

0.01128051417267848

In [19]:
accuracies

array([0.86875, 0.86   , 0.87875, 0.8625 , 0.86   , 0.8575 , 0.86625,
       0.84875, 0.84   , 0.8775 ])

In [21]:
accuracies.mean()

0.8619999999999999